In [5]:
import geemap 
import os 
import ee
import numpy as np

In [6]:
# Applies scaling factors.
def applyScaleFactors(image):
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermalBand = image.select('ST_B6').multiply(0.00341802).add(149.0)
  return image.addBands(opticalBands, None, True) \
              .addBands(thermalBand, None, True)

In [7]:
ee.Initialize()

## Draw map centered on Ivory Coast
- We can draw a region of interest to spatially filter the data 

In [22]:
# Parameters
collection = {'name': 'LANDSAT/LE07/C02/T1_L2', 'year range': [1999, 2022]} # Landsat 7
# collection = {'name': 'LANDSAT/LC08/C02/T1_L2', 'year range': [2013, 2023]} # Landsat 8
# collection = {'name': 'LANDSAT/LC09/C02/T1_L2', 'year range': [2021, 2023]} # Landsat 9
# collection = {'name': 'COPERNICUS/S2_SR_HARMONIZED', 'year range': [2017, 2023]} # Sentinel 2A

In [23]:
map = geemap.Map(center=(7.5, -5.5), zoom=6)
region = ee.Geometry.BBox(-9.66, 3.83, -1.28, 11.1)
map

Map(center=[7.5, -5.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(…

In [37]:
visualization = {
  'bands': ['SR_B3', 'SR_B2', 'SR_B1'],
  'min': 0.0,
  'max': 0.3,
}

out_dir = 'D:/Books/geopy-notebooks/images'

for year in np.arange(collection['year range'][0], collection['year range'][1]):
    map = geemap.Map()
    dataset = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
        .filterDate(str(year)+'-01-01', str(year)+'-12-31') \
        .filterBounds(region) \
        .filter(ee.Filter.lt('CLOUD_COVER',40)) \
        .map(applyScaleFactors).mosaic()

    # dataset = dataset
    
    map.centerObject(region, zoom=7)
    map.addLayer(dataset, visualization, 'Annual composite ' +str(year))
    
    features = geemap.fishnet(region, rows=10, cols=10)
    map.addLayer(features, {}, 'Grids')
    geemap.download_ee_image_tiles(dataset, features, out_dir, prefix="landsat_"+str(year), crs="EPSG:4326")

ImportError: Please install geedim using `pip install geedim` or `conda install -c conda-forge geedim`

In [32]:
visualization = {
  'bands': ['SR_B3', 'SR_B2', 'SR_B1'],
  'min': 0.0,
  'max': 0.3,
}

dataset = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
    .filterDate('2000-01-01', '2000-12-31')  \
    .filter(ee.Filter.lt('CLOUD_COVER', 50)) \
    .filterBounds(region) \
    .map(applyScaleFactors).mosaic()

map.centerObject(region, zoom=7)
map.addLayer(dataset, visualization, 'RGB')
map

Map(bottom=16003.0, center=[7.461496000493875, -5.469999999999933], controls=(WidgetControl(options=['position…

In [52]:
region = ee.Geometry.BBox(-9.66, 3.83, -1.28, 11.1)
image = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2').filterBounds(region) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)).median()

visualization = {
  'bands': ['SR_B3', 'SR_B2', 'SR_B1'],
  'min': 0.0,
  'max': 0.3,
}

map = geemap.Map(center=(7.5, -5.5), zoom=6)

map.addLayer(image, visualization, 'RGB')
map

Map(center=[7.5, -5.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(…

In [5]:
visualization = {
  'min': 0.0,
  'max': 0.3,
  'bands': ['B4', 'B3', 'B2'],
}

map.setCenter(-5.5, 7.5, 7)
map.addLayer(dataset.median(), visualization, 'RGB')
map

Map(bottom=4335.0, center=[7.5, -5.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=…

In [10]:
image = dataset.median()
image_clip = image.clip(roi)

filename = 'D:/Books/geopy-notebooks/images/ndvi_median.tif'

geemap.ee_export_image(image_clip, filename=filename, region=roi, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to D:\Books\geopy-notebooks\images\ndvi_median.tif
